# Query that returns both text(name) and code(ID) from JSON-stat
## Example HS-codes in foreign trade

### Import libraries
Use [pyjstat](https://pypi.org/project/pyjstat/) library for JSON-stat and pandas. Pandas is loaded as part of av pyjstat

In [1]:
from pyjstat import pyjstat
import requests

### URL with the table's metadata, where we post the query

In [2]:
tabid = "08799" # 
lang = "en" # language code can also be "no"
POST_URL = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabid # 'https://data.ssb.no/api/v0/en/table/08799'

### Query, can be taken from the console: 
Import / export all commodity codes (HS) to US latest month, ca. 65.000 cells. Max limit for one query in PxWebApi is 800.000 cells, incl. empty cells.

In [3]:
json_q = {
  "query": [
    {
      "code": "Varekoder",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ImpEks",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Land",
      "selection": {
        "filter": "item",
        "values": [
          "US"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Mengde1",
          "Verdi",
          "Mengde2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "top",
        "values": [
          "1"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}


### Post query

In [4]:
res = requests.post(POST_URL, json=json_q)

### Read JSON-stat result using the library pyjstat
Saved it as dataset ds.

In [5]:
ds = pyjstat.Dataset.read(res.text)

In [6]:
type(ds)

pyjstat.pyjstat.Dataset

Check dataset ds, large

In [7]:
# ds

### Get some main metadata from the JSON-stat dataset

In [8]:
title = ds['label']
print(title)

08799: External trade in goods, by commodity number, imports/exports, country, contents and month


Last update as GMT

In [9]:
last_update = ds['updated']
print(last_update)

2025-05-15T06:00:00Z


Get source

In [10]:
source = ds['source']
print(source)

Statistics Norway


New metadata in JSON-stat2, May 2025

In [11]:
heading = ds['extension']['px']['heading']
print(heading)

['ContentsCode', 'Tid']


In [12]:
stub = ds['extension']['px']['stub']
print(stub)

['Varekoder', 'ImpEks', 'Land']


Footnotes

In [13]:
if 'note' in ds:
	note = ds['note']
	print(note)

["Information about the validity and annual changes in the commodity numbers can be found at   <a target='_top' href='https://www.ssb.no/en/utenriksokonomi/varefortegnelse/commodity-list-external-trade'>Commodity List External Trade</a>.\n<br>\n\nFrom 2006 on the following countries had their belonging to continent changed - Cyprus from Asia to Europe, and Armenia, Georgia, Kyrgyzstan, Kazakhstan, Tajikistan, Turkmenistan and Uzbekistan from Europe to Asia. However, in the selections of countries in the pull-down menu for European/Asian countries it is the present classification that will be shown for the whole time period (Cypros belonging to Europe and the other countries to Asia).\n\n<br /><br />\nStatistics Norway do not publish figures for the trade region previously Comecon  after 2011. Former Comecon (only European Comecon countries in the Norwegian trade statistics): Armenia, Azerbaijan, Bulgaria, Estonia, Georgia, Belarus, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Moldova, Po

_Role_ gives some shortcuts

In [14]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'geo': ['Land'], 'metric': ['ContentsCode']})


### Make dataframes
We have to make two dataframes, one with text and one with id. Pyjstat returns 'label' by default.

In [15]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [16]:
hstrade.head()

,commodity number,imports/exports,country,contents,month,value
0,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Quantity 1 (Q1),2025M04,0
1,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Value (NOK),2025M04,0
2,"Imports of unspecified goods, with duty (1988-...",Imports,United States of America,Quantity 2 (Q2),2025M04,0
3,"Imports of unspecified goods, with duty (1988-...",Exports,United States of America,Quantity 1 (Q1),2025M04,0
4,"Imports of unspecified goods, with duty (1988-...",Exports,United States of America,Value (NOK),2025M04,0


In [17]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,00000011_1988,1,US,Mengde1,2025M04,0
1,00000011_1988,1,US,Verdi,2025M04,0
2,00000011_1988,1,US,Mengde2,2025M04,0
3,00000011_1988,2,US,Mengde1,2025M04,0
4,00000011_1988,2,US,Verdi,2025M04,0


### Make new column with ID and label concatinated

In [18]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['commodity number']

In [19]:
hstrade.columns

Index(['commodity number', 'imports/exports', 'country', 'contents', 'month',
       'value', 'hstrade_combi'],
      dtype='object')

### Make new dataframe with only the colums we want in new order. Mark double [[ ]]

In [20]:
hstrade_new = hstrade[['hstrade_combi', 'imports/exports', 'country', 'contents', 'month',
       'value']]

In [21]:
hstrade_new.tail()

,hstrade_combi,imports/exports,country,contents,month,value
80413,"99999999_1988 Total, all commodities not for p...",Imports,United States of America,Value (NOK),2025M04,174702436
80414,"99999999_1988 Total, all commodities not for p...",Imports,United States of America,Quantity 2 (Q2),2025M04,0
80415,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Quantity 1 (Q1),2025M04,90970447
80416,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Value (NOK),2025M04,572834089
80417,"99999999_1988 Total, all commodities not for p...",Exports,United States of America,Quantity 2 (Q2),2025M04,0
